In [7]:
# Maximizing Damage and Stats for a D&D fighter 
# Armor class for enemies has been set to 13. Chance to hit = 1 - (AC-bonus_to_hit)/20
# Reminder: Crits only double damage on dice, not the modifiers.

import cvxpy as cp

enemyAC = 13
prof_bonus = 3
# Assume light leather armor

attributes = cp.Variable(6, integer=True) 
standard_array = cp.Variable((6,6), boolean=True)
feat_bonus = cp.Variable(6, integer=True)
has_proficiency = cp.Variable(18, integer=True)
race = cp.Variable(14, boolean=True)
background = cp.Variable(13, boolean = True)

constraints = []

#15 – 14 – 13 – 12 – 10 – 8
for i in range(6):
    constraints.append(attributes[i] == 15*standard_array[i,0] + 14*standard_array[i,1] + 13*standard_array[i,2] + 12*standard_array[i,3] + 10*standard_array[i,4] + 8*standard_array[i,5])
    constraints.append(cp.sum(standard_array[:,i]) ==1)
    constraints.append(cp.sum(standard_array[i,:]) ==1)

constraints.append(cp.sum(race)==1)
constraints.append(cp.sum(background)==1)
constraints.append(cp.sum(feat_bonus)==2+race[9]*2)
    
# Ability Score constraints 8 <= ability <= 18
for i in range(6):
    constraints.append(attributes[i] >= 8)
    constraints.append(attributes[i] <= 18)
    constraints.append(attributes[i] + feat_bonus[i] <= 18)
    
# Nonnegativity constraints
for i in range(6):
    constraints.append(attributes[i] >= 0)
    constraints.append(feat_bonus[i] >= 0)
    constraints.append(feat_bonus[i] <= 2)

# Proficiency Bonuses
constraints.append(has_proficiency[0] == background[3] + 1)
constraints.append(has_proficiency[1] == background[4])
constraints.append(has_proficiency[2] == background[9])
constraints.append(has_proficiency[3] == background[8] + background[10] + background[11] + 1)
constraints.append(has_proficiency[4] == background[1] + background[2])
constraints.append(has_proficiency[5] == background[7] + background[9])
constraints.append(has_proficiency[6] == background[0] + background[5])
constraints.append(has_proficiency[7] == race[9] + background[11])
constraints.append(has_proficiency[8] == 0)
constraints.append(has_proficiency[9] == background[6])
constraints.append(has_proficiency[10] == 0)
constraints.append(has_proficiency[11] == race[3] + race[4] + race[5] + background[10])
constraints.append(has_proficiency[12] == background[3])
constraints.append(has_proficiency[13] == background[5] + background[7])
constraints.append(has_proficiency[14] == background[0] + background[6])
constraints.append(has_proficiency[15] == background[1] + background[12])
constraints.append(has_proficiency[16] == background[2] + background[12])
constraints.append(has_proficiency[17] == background[4] + background[8])


#Constraints of proficiency bonus
for i in range(18):
    constraints.append(has_proficiency[i] >= 0)
    constraints.append(has_proficiency[i] <= 1)

# Calculated ability scores
str = attributes[0] + race[0]*2 + race[2]*2 + race[9]*2 + race[12]  + feat_bonus[0]
dex = attributes[1] + race[1]*2 + race[3]*2 + race[4]*2 + race[5]*2 + race[6] + race[10]*2 + race[11]*2 + race[12] + feat_bonus[1]
con = attributes[2] + race[2]*2 + race[7] + race[9] + race[11] + race[12] + feat_bonus[2]
int = attributes[3] +  race[4] + race[6]*2 + race[7]*2 + race[12] + race[13] + feat_bonus[3]
wis = attributes[4] + race[1] + race[5] + race[12] + feat_bonus[4]
char = attributes[5] + race[0] + race[3] + race[8]*2 + race[10] + race[12] + race[13]*2 + feat_bonus[5]


strmod = (str - 10) / 2
dexmod = (dex - 10) / 2
conmod = (con - 10) / 2
intmod = (int - 10) / 2
wismod = (wis - 10) / 2
charmod = (char - 10) / 2


#Skill weights determined by this website here: Low usefullness gets 0.2 mod, med gets 0.4, high gets 0.6, essential gets 0.8
# Use website here: https://gamerant.com/dungeons-dragons-best-worst-useful-ability-skills-ranked/#deception-charisma
TURNS = 3
 # Fighter Champion; Archery - Heavy Cross-bow
#damage_output = (TURNS+1)*1.6*(5.5+dexmod)+(TURNS+1)*2*crit*5.5
 # Fighter Champion; Dueling - Long Sword VS Rapier
#damage_output = (TURNS+1)*1.4*(4.5+2+strmod)+(TURNS+1)*2*crit*4.5
#damage_output = (TURNS+1)*1.4*(4.5+2+dexmod)+(TURNS+1)*2*crit*4.5
 # Fighter Champion; Great Weapon Fighting - Greatsword
#damage_output = (TURNS+1)*1.4*(8.34+strmod)+(TURNS+1)*2*crit*7
 # Fighter Champion; Two-Weapon Fighting Str then Dex Build
#damage_output = (TURNS+1)*1.4*(3.5+strmod)+2*crit*3.5 + (TURNS)*.7*(3.5+strmod)+2*crit*3.5
#damage_output = (TURNS+1)*1.4*(3.5+dexmod)+2*crit*3.5 + (TURNS)*.7*(3.5+dexmod)+2*crit*3.5

# Fighter Battle Master; Archery - Heavy Cross-bow (one with SD)
#damage_output = TURNS*.8*(10+dexmod)+TURNS*crit*10 + (TURNS+2)*.8*(5.5+dexmod)+(TURNS+2)*crit*5.5
 # Fighter Battle Master; Dueling - Long Sword VS Rapier (one with Feinting SD)
#damage_output = TURNS*.91*(9+2+strmod)+TURNS*2*crit*9 + (TURNS+2)*.7*(4.5+2+strmod)+(TURNS+2)*crit*4.5
#damage_output = TURNS*.91*(9+2+dexmod)+TURNS*2*crit*9 + (TURNS+2)*.7*(4.5+2+dexmod)+(TURNS+2)*crit*4.5
 # Fighter Battle Master; Great Weapon Fighting - Greatsword (one with Feinting SD)
#damage_output = TURNS*.91*(12.84+strmod)+TURNS*2*crit*11.5 + (TURNS+2)*.7*(8.34+strmod)+(TURNS+2)*crit*7
 # Fighter Battle Master; Two-Weapon Fighting (one with SD) Str then Dex Build
#damage_output = TURNS*.7*(8+strmod)+TURNS*crit*8 + ((2*TURNS)+2)*.7*(3.5+strmod)+((2*TURNS)+2)*crit*3.5
damage_output = TURNS*.7*(8+dexmod)+TURNS*crit*8 + ((2*TURNS)+2)*.7*(3.5+dexmod)+((2*TURNS)+2)*crit*3.5
# CONTROL
#damage_output = 0

AC = 11 + dexmod

hitpoints_per_level = 6 + conmod + race[1]

skillweight = 0.6 * (dexmod  + prof_bonus * has_proficiency[0]) #acrobatics # 7
skillweight += 0.4 * (wismod + prof_bonus * has_proficiency[1]) #Animal handling # 14
skillweight += 0.4 * (intmod + prof_bonus * has_proficiency[2]) # Arcana # 11
skillweight += 0.6 * (strmod + prof_bonus * has_proficiency[3]) # Atheletics # 7 two number 7s
skillweight += 0.6 * (charmod + prof_bonus * has_proficiency[4]) # Deception # 6
skillweight += 0.4 * (intmod + prof_bonus * has_proficiency[5]) # History # 10
skillweight += 0.8 * (wismod + prof_bonus * has_proficiency[6]) # Insight # 4
skillweight += 0.4 * (charmod + prof_bonus * has_proficiency[7]) # Intimidation # 12
skillweight += 0.6 * (intmod + prof_bonus * has_proficiency[8]) # Investigation # 5
skillweight += 0.2 * (wismod + prof_bonus * has_proficiency[9]) # Medicine # 13
skillweight += 0.4 * (intmod + prof_bonus * has_proficiency[10]) # Nature 9
skillweight += 0.8 * (wismod + prof_bonus * has_proficiency[11]) # Perception 1
skillweight += 0.2 * (charmod + prof_bonus * has_proficiency[12]) #Performance high
skillweight += 0.8 * (charmod + prof_bonus * has_proficiency[13]) # Persuasion 2
skillweight += 0.2 * (intmod + prof_bonus * has_proficiency[14]) # Religion 17
skillweight += 0.6 * (dexmod + prof_bonus * has_proficiency[15]) #SOF 8
skillweight += 0.8 * (dexmod + prof_bonus * has_proficiency[16]) # stealth 3
skillweight += 0.2 * (wismod + prof_bonus * has_proficiency[17]) #survuval 15

obj_func= damage_output + AC + 2.5*hitpoints_per_level + skillweight
                   
problem = cp.Problem(cp.Maximize(obj_func), constraints)

problem.solve(solver=cp.GUROBI,verbose = False)

print("obj_func =", obj_func.value)
print("Attributes: ", str.value, ", ", dex.value, ", ", con.value, ", ", int.value, ", ", wis.value, ", ", char.value)
print("Race: ", race.value)
print("Background: ", background.value)
print("proficiencies =", has_proficiency.value)

# Races in Order:
#0 Dragonborn
#1 Hill Dwarf
#2 Mountain Dwarf
#3 Dark Elf
#4 High Elf
#5 Wood Elf
#6 Forest Gnome
#7 Rock Gnome
#8 Half-Elf
#9 Half-Orc
#10 Lightfoot Halfling
#11 Stout Halfling
#12 Human
#13 Tiefling

# Backgrounds in Order: 
#0 Acolyte
#1 Charlatan
#2 Criminal
#3 Entertainer
#4 Folk Hero
#5 Guild Artisan
#6 Hermit
#7 Noble
#8 Outlander
#9 Sage
#10 Sailor
#11 Soldier
#12 Urchin

NameError: name 'crit' is not defined